---
title: Sequence Containers
abstract: |
    Encapsulation is a fundamental principle of object-oriented programming that promotes modularity and data protection by bundling data and the methods that operate on it within a single unit. Standard library containers such as array, vector, list, set, and map exemplify encapsulation by exposing well-defined interfaces while hiding internal implementation details. Encapsulation also facilitates resource management strategies like move semantics, where rvalue references and universal references allow efficient transfer and reuse of resources. This notebook introduces the basic idea of encapsulation using sequence containers such as vector and list.
math:
    '\abs': '\left\lvert #1 \right\rvert'
    '\norm': '\left\lVert #1 \right\rVert'
    '\Set': '\left\{ #1 \right\}'
    '\set': '\operatorname{set}'   
    '\mc': '\mathcal{#1}'
    '\M': '\boldsymbol{#1}'
    '\R': '\mathsf{#1}'
    '\RM': '\boldsymbol{\mathsf{#1}}'
    '\op': '\operatorname{#1}'
    '\E': '\op{E}'
    '\d': '\mathrm{\mathstrut d}'
    '\SFM': '\operatorname{SFM}'
    '\utag': '\stackrel{\text{(#1)}}{#2}'
    '\uref': '\text{(#1)}'
    '\minimal': '\operatorname{minimal}'
skip_execution: true
---

In [ ]:
from __init__ import *
%reload_ext divewidgets

In [ ]:
%%cpp
#include "utility.hpp"

In [ ]:
if not input('Load JupyterAI? [Y/n]').lower()=='n':
    %reload_ext jupyter_ai

## Motivation

Encapsulation is the technique of capturing resources such as variables (properties) and functions (methods) into an entity (object). This lays the foundation for object‑oriented programming (OOP).

To motivate the concept, consider the problem of enumerating [combinations](https://en.wikipedia.org/wiki/Combination) in [combinatorics](https://en.wikipedia.org/wiki/Combinatorics):

::::{prf:definition} combination
:label: def:combination

A $k$-combination $S$ of $n$ items, say $\Set{0, \dots, n-1}$, is a selection of $k$ items from the set.

::::

::::{prf:example}

For $3$ items $\Set{0, 1, 2}$, we have:
- one $0$-combination:
  > $\Set{}$
- three $1$-combinations:
  > $\Set{0}, \Set{1}, \Set{2}$
- three $2$-combinations:
  > $\Set{0, 1}, \Set{0, 2}, \Set{1, 2}$
- no $k$-combinations for other values of $k$.

::::

**How to define a function to compute combinations?**

A divide-and-conquer approach is as follows:

::::{prf:proposition}
:label: pro:combination

A $k$-combination $S$ of $n$ items satisfies the recurrence relation for $1\leq k\leq n$:

> $S\setminus \Set{n-1}$ is a combination of $n-1$ items with
>  - $k-1$ items if ${{\color{red}n-1}}\in S$; or
>  - ${\color{blue}k}$ items otherwise.

Reversing the above removal process gives a way of constructing a $k$-combination $S$ from a smaller combination $T$ of $n-1$ items with
- ${\color{blue}k}$ items; or
- $k-1$ items and add ${\color{red}n-1}$ to it.

If $0=k< n$, $S=\Set{}$. $S$ does not exists for other choices of $(n, k)$. E.g.,

$$
\overbrace{{\color{blue}\Set{0, 1}}}^{\mathclap{\text{\scriptsize ${\color{blue}2}$-combination of $\{0, 1\}$}}}, \underbrace{\Set{0, {\color{red}2}}, \Set{1, {\color{red}2}}}_{\mathclap{\text{\scriptsize $2$ added to $1$-combinations of $\{0, 1\}$}}}.
$$


::::

A $k$-combination in [](#def:combination) can be implemented as a vector container of indices from $\Set{0, \dots, n-1}$:

In [ ]:
%%cpp
vector<size_t>{0,1,2}

The collection of combinations can be a vector of a vector of indices:

In [ ]:
%%cpp
vector<vector<size_t> > {{0,1},{0,2},{1,2}}

To implements the divide-and-conquer approach in [](#pro:combination):

::::{code} cpp
:label: code_combination1
:linenos:
:caption: Returning a `std::vector` of $k$-combinations of $n$ items $\Set{0,1, \dots, n-1}$.
/**
 * @brief Generates all k-combinations of the set {0, 1, ..., n-1}.
 *
 * This function uses recursion to build combinations by either:
 * - excluding the last element (n-1) and generating k-combinations of the first n-1 elements
 * - including the last element (n-1) and generating (k-1)-combinations of the first n-1 elements,
 *   then appending n-1 to each of those to form a full k-combination.
 *
 * @param n The size of the set to choose from.
 * @param k The number of elements to choose.
 * @return A vector of vectors, each representing a k-combination of the set {0, ..., n-1}.
 */
std::function<std::vector<vector<size_t> >(const size_t, const size_t)>
combination=[&combination](const size_t n, const size_t k) {
    vector<vector<size_t> > output; // base case when k is not in {0, ..., n}.
    if (k<=n) {
        if (k) {
            // Case 1: k-combinations from the first n-1 elements
            for (auto &subset : combination(n-1, k)) output.push_back(subset);
            
            // Case 2: (k-1)-combinations from the first n-1 elements, with n-1 added
            for (auto &subset : combination(n-1, k-1)) {
                subset.push_back(n-1);
                output.push_back(subset);
            }
        } else output.push_back(vector<size_t>()); // base case: only one combination of size 0 (empty set)
    } // Hidden base case: No combinations if k is not in the range {0, ..., n}
    return output;
};
::::

In [ ]:
%%cpp
std::function<std::vector<vector<size_t> >(const size_t, const size_t)>
combination=[&combination](const size_t n, const size_t k) {
    vector<vector<size_t> > output;
    if (k<=n) {
        if (k) {
            for (auto &subset : combination(n-1, k)) output.push_back(subset);
            for (auto &subset : combination(n-1, k-1)) {
                subset.push_back(n-1);
                output.push_back(subset);
            }
        } else output.push_back(vector<size_t>());
    }
    return output;
};

The following generates different combinations of $3$ items:

In [ ]:
%%cpp
combination(3, 0)

In [ ]:
%%cpp
combination(3, 1)

In [ ]:
%%cpp
combination(3, 2)

In [ ]:
%%cpp
combination(3, 3)

In [ ]:
%%cpp
combination(3, 4)

::::{exercise}
:label: ex:combination1

Why the following code does not return an empty vector?

::::

In [ ]:
%%cpp
combination(-1,0).empty()

YOUR ANSWER HERE

For instance we can compute ${22 \choose 11}$

Consider counting the number combinations using the [size](https://en.cppreference.com/w/cpp/container/vector/size.html) method:

```cpp
size_type size() const;
```

In [ ]:
%%ai
Explain in one line what does `const` mean for the method
--
size_type size() const;

For instance, the total number of $11$-combinations of $22$ items is:

In [ ]:
%%cpp
combination(22, 11).size()

The above requires about 6 seconds to compute. Similarly, the following computes the total number of $12$-combinations of $24$ items: (It takes about **30 seconds** to execute. You have been warned.)

In [ ]:
%%cpp
combination(24, 12).size()

**How to improve the performance?**

If the goal is to compute the number of combinations, we can do it simply by computing the binomial coefficient:

::::{prf:proposition}
:label: pro:binomial_coefficient

The number of $k$-combinations of $n$ items is 

$$
{n \choose k} := \frac{n!}{(n-k)!k!},
$$

which is called the [binomial coefficient](https://en.wikipedia.org/wiki/Binomial_coefficient).

::::

In [ ]:
%%cpp
unsigned long binomial_coefficient(const size_t n, const size_t k) {
    return k<=n? (k? binomial_coefficient(n-1, k) + binomial_coefficient(n-1, k-1) : 1uL) : 0uL;
}
binomial_coefficient(24, 12)

How to improve the performance of `combination`?

Thanks to encapsulation, we can easily improve our code by choosing a suitable object and its associated methods:

::::{code} cpp
:label: code_combination2
:linenos:
:caption: Returning a [`std::list`](https://en.cppreference.com/w/cpp/container/list.html) of $k$-combinations of $n$ items $\Set{0,1, \dots, n-1}$.
std::function<std::list<vector<size_t> >(const size_t, const size_t)>
combination=[&combination](const size_t n, const size_t k) {
    list<vector<size_t> > output;
    if (k<=n) {
        if (k) {
            output.splice(output.cend(), combination(n-1, k));
            list<vector<size_t> > subset_seq=combination(n-1, k-1);
            for (auto &subset : subset_seq) subset.push_back(n-1);
            output.splice(output.cend(), subset_seq);
        } else output.emplace_back(vector<size_t>());
    }
    return output;
};
::::

In [ ]:
%%cpp
std::function<std::list<vector<size_t> >(const size_t, const size_t)>
combination=[&combination](const size_t n, const size_t k) {
    list<vector<size_t> > output;
    if (k<=n) {
        if (k) {
            output.splice(output.cend(), combination(n-1, k));
            list<vector<size_t> > subset_seq=combination(n-1, k-1);
            for (auto &subset : subset_seq) subset.push_back(n-1);
            output.splice(output.cend(), subset_seq);
        } else output.push_back(vector<size_t>());
    }
    return output;
};

combination(3, 2)

The following call now completes in about 1 second instead of 6 seconds:

In [ ]:
%%cpp
combination(22, 11).size()

The following call completes in about 3-4 seconds instead of 30 seconds:

In [ ]:
%%cpp
combination(24, 12).size()

That is close to 10 times speed up!

In [](#code_combination1), we mainly used the method [`push_back`](https://en.cppreference.com/w/cpp/container/vector/push_back.html) to create each combination and the collection of all combinations. There are two overloads:

```cpp
void push_back( const T& value );
void push_back( T&& value );
```

Which overload is used and why?

In [ ]:
%%ai
Explain very briefly what are the differences between the two overloads:
--
void push_back( const T& value );
void push_back( T&& value );

In [](#code_combination2), we used the [`std::list`](https://en.cppreference.com/w/cpp/container/list.html) object to store the collection of combinations, and its method [`splice`](https://en.cppreference.com/w/cpp/container/list/splice.html) to concatenate lists of combinations:

```cpp
void splice( const_iterator pos, list& other );
void splice( const_iterator pos, list&& other );
void splice( const_iterator pos, list& other, const_iterator it );
void splice( const_iterator pos, list&& other, const_iterator it );
void splice( const_iterator pos, list& other,
             const_iterator first, const_iterator last );
void splice( const_iterator pos, list&& other,
             const_iterator first, const_iterator last );
```

Again, which overload is used and why?

In [ ]:
%%ai
Explain very briefly what are the differences between the two overloads:
--
void splice( const_iterator pos, list& other );
void splice( const_iterator pos, list&& other );
void splice( const_iterator pos, list& other, const_iterator it );
void splice( const_iterator pos, list&& other, const_iterator it );
void splice( const_iterator pos, list& other,
             const_iterator first, const_iterator last );
void splice( const_iterator pos, list&& other,
             const_iterator first, const_iterator last );

We also used the method [`emplace_back()`](https://en.cppreference.com/w/cpp/container/list/emplace_back.html) in [](#code_combination2) in place of `push_back(vector<size_t>())` in [](#code_combination1) to add an empty combination, without constructing the combination explicitly:

```cpp
template< class... Args >
reference emplace_back( Args&&... args );
```

What does the `...` mean and why is it used?

In [ ]:
%%ai
Explain very briefly the following syntax:
--
template< class... Args >
reference emplace_back( Args&&... args );

In [ ]:
%%ai
Why construct in-place using `emplace_back` instead of `push_back`?

While we can reuse code easily with OOP, we need to know how to choose the object, its methods, and their overloads. That demands a deeper understanding of the design of the object and how its resources are managed. We will take a closer look at `std::vector` and `st::list`.

## Referencing Resources

To understand the difference between `&&` and `&`, recall the following code from a previous lecture. What is the value of `a` after executing the following code?

In [ ]:
%%cpp
auto a=1, &b=a;
b+=1;
a

To see why `a` is `2` instead of `1`, we can print the addresses of `a` and `b` and show that the addresses are the same:

In [ ]:
%%cpp
auto a=1, &b=L(a);
L(b)+=1;
a

`L` is a macro

```cpp
#define L(x) loc(x, std::string("(")+std::string(type<decltype(x)>())+") "+#x)
```

which expands to a call to the function `loc` in the header file ["utility.hpp"](./utility.hpp). The function

- decorates a value by printing its address, but
- preserves the value so that the behavior of the original code remains unchanged.

**How to properly pass an argument by reference to implement the `loc` function?**

### Lvalue Reference

The following implementation passes the argument and return it by [lvalue reference](https://en.cppreference.com/w/cpp/language/reference.html#Lvalue_references):

In [ ]:
%%cpp
auto &loc1(auto &x) {
    cout << format("@{:p}\n", static_cast<void *>(&x));
    return x;
}

It appears to work:

In [ ]:
%%cpp
auto a=1, &b=loc1(a);
loc1(b)+=1;
a

::::{exercise}
:label: ex:loc1:const

Why does `loc1` fail for constant arguments?

```cpp
const auto a=1, &b=loc1(a);
loc1(b)
```

:::{hint}

See [const_cast](https://en.cppreference.com/w/cpp/language/const_cast.html).

:::

::::

YOUR ANSWER HERE

Instead of using `const_cast`, it is safer to implement the following fix:

In [ ]:
%%cpp
auto &loc1(auto &x) {
    cout << format("@{:p}\n", static_cast<const void *>(&x));
    return x;
}

`loc1` now works for constants:

In [ ]:
%%cpp
const auto a=1, &b=loc1(a);
loc1(b)

In [ ]:
%%ai
Explain briefly when it is essential to use const_cast instead of static_cast
for constant pointers.

Consider the following direct initialization:

In [ ]:
%%cpp
auto u(vector<int>{1, 2, 3});

**Does `u` reuse the memory allocated to the rvalue `vector<int>{1, 2, 3}`?**

::::{exercise}
:label: ex:loc1

Why using `loc1` as follows leads to an error?

```cpp
auto u(loc1(vector<int>{1, 2, 3}));
loc1(u);
```

As the following program shows, `loc1` appears to work for C-string but fails for the compound literal for `char`. Why?

::::

In [ ]:
%%cpp
loc1("a")        // works?!

In [ ]:
%%cpp
// loc1((char[]){'a');  // fails: expects an lvalue for 1st argument
// loc1(static_cast<char * const>((char[]){'a', 0})); // making it constant also fail
(char[]){'a', 0}            // isn't it just "a"?

YOUR ANSWER HERE

A potential fix is to add `const` qualifier to the parameter `x`:

In [ ]:
%%cpp
auto &loc2(const auto &x) {
    cout << format("@{:p}\n", static_cast<const void *>(&x));
    return x;
}

In [ ]:
%%cpp
auto u(loc2(vector<int>{1, 2, 3}));
loc2(u)

::::{exercise}
:label: ex:loc2

Why does the following code fail?

```cpp
auto u(loc2(vector<int>{1, 2, 3}));
loc2(u).push_back(4);
```

::::

YOUR ANSWER HERE

### Rvalue Reference

A potential fix is to pass the argument by [rvalue reference](https://en.cppreference.com/w/cpp/language/reference.html#Rvalue_references) using `&&` instead of `&`:

In [ ]:
%%cpp
auto &loc3(auto &&x) {
    cout << format("@{:p}\n", static_cast<const void *>(&x));
    return x;
}

In [ ]:
%%cpp
auto u(loc3(vector<int>{1, 2, 3}));
loc3(u).push_back(4);
u

It seems to work now, right?

In [ ]:
%%cpp
loc3(1) = 0;   // works?!
1              // 1 is now 0?!

::::{caution}  It doesn't work. Why?
:class: dropdown

`loc3` returns an lvalue instead, which is assignable even for simple integer type. In other words, `loc3` changed the behavior of the original code.

::::

::::{exercise}
:label: ex:rvalue_ref1
Why does the following code not change `1` to `2`?

::::

In [ ]:
%%cpp
[](int &&x) {x=0;}(1);  // change 1 to 0
auto x=1                // x is 0?

YOUR ANSWER HERE

How about returning the output by rvalue reference as well?

In [ ]:
%%cpp
template <class T>
T &&loc4(T &&x) {
    cout << format("@{:p}\n", static_cast<const void *>(&x));
    return x;
}

::::{caution}

We used `template` instead of `auto` above because `auto &&loc4(auto &&x) { ... }` does not work for the ROOT kernel, similar to [this issue](https://github.com/jupyter-xeus/xeus-cling/issues/40).

::::

::::{exercise}
:label: ex:loc4

Why does the following code fail?

```cpp
auto u(loc4(vector<int>{1, 2, 3}));
```

::::

YOUR ANSWER HERE

A potential fix is to cast `x` to an rvalue reference using [`std::move(x)`](https://en.cppreference.com/w/cpp/utility/move.html):

In [ ]:
%%cpp
template <class T>
T &&loc5(T &&x) {
    cout << format("@{:p}\n", static_cast<const void *>(&x));
    return std::move(x);
}

It appears to work now:

In [ ]:
%%cpp
auto u(loc5(vector<int>{1, 2, 3}));
// loc5(1) = 0;   // fails: expression is not assignable

Does it really work? `loc5` fails to handle lvalue:

```cpp
auto a=1;
loc5(a);
```

The error message is:

::::{code}
:label: err_loc5
:linenos:
:emphasize-lines: 2
error: non-const lvalue reference to type 'int' cannot bind to a temporary of type 'typename
std::remove_reference<int &>::type' (aka 'int')
    return std::move(x);
           ^~~~~~~~~~~~
::::

### Universal Reference

A potential fix is to use `static_cast` instead of `std::move`:

In [ ]:
%%cpp
template <class T>
T &&loc6(T &&x) {
    cout << format("@{:p}\n", static_cast<const void *>(&x));
    return static_cast<T &&>(x);
}

In [ ]:
%%cpp
auto u(loc6(vector<int>{1, 2, 3}));
loc6(u)

But how can that make a difference? According to [the documentation](https://en.cppreference.com/w/cpp/utility/move.html):
> It (`std::move`) is exactly equivalent to a static_cast to an rvalue reference type.

The same fix does not work below:

In [ ]:
%%cpp
int a=1;
[](int &&b) { return static_cast<int &&>(b); }
// (a);  // fails: expects an rvalue for 1st argument

An important clue comes from the [error message of `loc5(a)`](#err_loc5):

```cpp
std::remove_reference<int &>::type' (aka 'int')
```

The return type from `loc5` is an lvalue, not an rvalue!

::::{caution} Universal reference
:class: dropdown

For parameters of function templates, `&&` is a [forwarding/universal reference](https://en.cppreference.com/w/cpp/language/reference.html#Forwarding_references) instead of an rvalue reference. It simply forward the argument, preserving its value category:

- For the code `loc5(a);`, `a` is passed by lvalue reference, i.e., `T &&` becomes `int &`, which is also the return type by the function declaration `T &&loc5(T &&x)`. The return lvalue reference is non-const and therefore cannot bind to a temporary `std::move(x)`.
- `loc6(a);` works because `static_cast<T &&>(x)` instantiates to `static_cast<int &>(x)`, which is an lvalue reference instead of a temporary!

::::

We can use [`std::forward<T>`](https://en.cppreference.com/w/cpp/utility/forward.html) instead of `static_cast<T &&>` to make it less confusing:

::::{code} cpp
:label: code_loc
:caption: The implementation of `loc` in ["utility.hpp"](./utility.hpp).
/**
 * @brief Logs the memory address of a variable along with an optional label.
 * 
 * This function is useful for educational purposes, especially when tracking
 * object lifetimes, locations in memory, or verifying move semantics.
 * 
 * @tparam T Type of the variable (supports universal references).
 * @param x The variable to log.
 * @param label Optional label to identify the variable in the output.
 * @return T&& Perfectly forwarded variable.
 */
template <class T>
T &&loc(T &&x, const string &label="") {
    cout << format("{} @{:p}\n",
        label,
        static_cast<const void*>(&x));
    return std::forward<T>(x);
}
::::

In [ ]:
%%cpp
auto u(loc(vector<int>{1, 2, 3}));
loc(u)

**Back to our original endeavor, does `u` reuse the memory allocated to the rvalue `vector<int>{1, 2, 3}`?**

It seems that the rvalue `vector<int>{1, 2, 3}` occupies a different memory location than the lvalue `u` does. What about copy initializations?

In [ ]:
%%cpp
auto u=LB((vector<int>{1, 2, 3}));
auto v=LB(u);
LB(v)

In [ ]:
%%ai
Explain very briefly how direct initialization in C++ is different from
copy initialization and list initialization?

We are now ready to dive into how objects are constructed using different constructors and initializations.

## Vector Construction

There many [constructors](https://en.cppreference.com/w/cpp/container/vector/vector.html) available for constructing vectors. Unlike other methods, constructors have no names but can be called in various ways to create rvalues or to initialize variables.

### Converting Constructors

Constructors that take a single argument and is not marked as [explicit](https://en.cppreference.com/w/cpp/language/explicit.html) are called [converting constructors](https://en.cppreference.com/w/cpp/language/converting_constructor.html). These constructors can be used for implicit conversions or copy initialization.

#### Initializer‑List Constructor

```cpp
vector( std::initializer_list<T> init,
        const Allocator& alloc = Allocator() );
```
adds elements from a (braced) [initializer list](https://en.cppreference.com/w/cpp/utility/initializer_list/initializer_list.html).

In [ ]:
%%cpp
vector<size_t> {0, 1, 2}     // vector constructed with braced initializer list

In [ ]:
%%cpp
vector<size_t> u {0, 1, 2}   // list initialization

In [ ]:
%%cpp
vector<size_t> u({0, 1, 2})  // direct initialization with braced initializer list

In [ ]:
%%cpp
vector<size_t> u={0, 1, 2}   // copy-list initialization

To use an initializer list explicitly:

In [ ]:
%%cpp
initializer_list<size_t> ilist {0, 1, 2};
(vector<size_t>(ilist))  // vector constructed with initializer list

In [ ]:
%%cpp
vector<size_t> u(init_list)  // direct initialization with initializer list

In [ ]:
%%ai
Explain very briefly how is copy-list initialization different from
list initialization for `std::vector`.

#### Move Constructor

```cpp
vector( vector&& other );
```
moves elements from another vector:

In [ ]:
%%cpp
vector<int>(vector<int>{1, 2, 3}) // vector constructed with elements moved from other

In [ ]:
%%cpp
auto u(vector<int>{1, 2, 3})      // direct initialization

In [ ]:
%%cpp
auto u=vector<int>{1, 2, 3}       // copy initialization

To see that elements are not duplicated:

In [ ]:
%%cpp
auto u(LB(L((vector<int>{1, 2, 3}))));
L(LB(u))

Even though the lvalue `u` and rvalue `vector<int>{1, 2, 3}` have different memory locations, they begin storing their elements at the same location, i.e., the lvalue reuses the memory allocated to the rvalue.

:::::{seealso} What is `LB`?

`LB` is a macro defined as

```cpp
#define LB(x) loc_begin(x, std::string("(")+std::string(type<decltype(x)>())+") "+#x)
```

which expands to a call to `loc_begin` in ["utility.hpp"](./utility.hpp) to locate the beginning of the vector:

::::{code} cpp
// Overload for containers with .cbegin()
template <class T>
std::enable_if_t<has_begin<T>::value, T&&>
&&loc_begin(T &&x, const string &label="") {
    cout << format("{} begin@{:p}\n", label, static_cast<const void*>(&*(x.cbegin())));
    return std::forward<T>(x);
}
::::
:::::

To see that the elements are moved:

In [ ]:
%%cpp
vector<size_t> u {1,2,3}, v(std::move(LB(u))); // move elements of `u` to `v`
LB(u).empty()? LB(v): throw runtime_error("Elements of `u` should be moved!")

`std::move` casts `u` into an rvalue, which is then passed to the move constructor: 
- The move constructor steals the resources from `u`, making it empty and
- the address of its first element becomes `nullptr`.

#### Copy Constructor

```cpp
vector( const vector& other );
```
can be used to copy elements from another vector:

In [ ]:
%%cpp
vector<size_t> u {1,2,3};
(vector<size_t>(u))  // vector constructed with elements copied from other

In [ ]:
%%cpp
auto v(u)  // direct initialization

In [ ]:
%%cpp
auto v=u  // copy initialization

To see that the elements are duplicated:

In [ ]:
%%cpp
LB(u), LB(v)

`v` is equal to `u` but it does not begin at the same location as `u`.

::::{seealso} Object copying

- `v` is said to be a [deep copy](https://en.wikipedia.org/wiki/Object_copying#Deep_copy) of `u` in the sense that `v` is a new container whose elements are copies of those of `u`.
- In contrast, a [shallow copy](https://en.wikipedia.org/wiki/Object_copying#Shallow_copy) reuses the elements of the original.

::::

Shallow copy is prone to mutability bug. For instance:

::::{exercise}
:label: ex:deep_copy

Explain why `s` at the end of the following code is not `"aba"`?

:::{hint}

The first call `LB(s)` uses the overload

```cpp
// Overload for C-style arrays
template <class T, size_t N>
T (&loc_begin(T (&x)[N], const string &label = ""))[N] {
    cout << format("{} begin@{:p}\n", label, static_cast<const void*>(&x[0]));
    return x;
}
```

but the second call `LB(t)` uses the overload

```cpp
// For pointers or decayed arrays
template <class T>
typename std::enable_if<std::is_pointer<T>::value, T>::type
loc_begin(T x, const string& label = "") {
    cout << format("{} begin@{:p}\n", label, static_cast<const void*>(x));
    return x;
}
```

:::

::::

In [ ]:
%%cpp
char s[]="aba";
auto t(LB(s));
LB(t)[1]='v';
s // aba?

YOUR ANSWER HERE

::::{exercise}
:label: ex:constructors1

Explain all the constructors involved in the following declaration.

::::

In [ ]:
%%cpp
auto u=vector<vector<size_t> > {{1}, {2, 3}}, v=u;
v

YOUR ANSWER HERE

### Explicit Constructors

#### Default Constructor

```cpp
vector() noexcept(noexcept(Allocator())) : vector(Allocator()) {}
```
takes no argument and returns an empty vector:

In [ ]:
%%cpp
vector<size_t>()

In [ ]:
%%cpp
vector<size_t> u

:::{caution}

`vector<size_t> u()` is also incorrect as it can be mistaken to be a function declaration instead of a direct initialization with the default initializer.

:::

In [ ]:
%%ai
Explain brielfy the motivation of `noexcept` in the constructor:
`vector() noexcept(noexcept(Allocator())) : vector(Allocator()) \{\}`

There are constructors with more than one arguments or marked as explicit, and therefore, cannot be used in copy initialization nor implicit conversion.

#### Fill Constructors

:::{code} cpp
:linenos:
explicit vector( size_type count, const Allocator& alloc = Allocator() );
vector( size_type count, const T& value, const Allocator& alloc = Allocator() );
::: 
fill a vector with `n` copies of `value`, if available, and `0` otherwise:

In [ ]:
%%cpp
vector<size_t>(2)           // vector constructed by the fill constructor

In [ ]:
%%cpp
vector<size_t> u(3, 4)      // direct initialization

::::{exercise} 
:label: ex:constructor2

Why `vector<size_t> v=2;` does not create a vector `v` same as `u`?

::::

YOUR ANSWER HERE

#### Range Constructor

```cpp
template<class InputIt> vector(InputIt first, InputIt last, const Allocator& a = Allocator())
```
copies elements within a range from the position `first` up to but excluding `last`:

In [ ]:
%%cpp
vector<size_t> u {0, 1, 2, 3};
vector<size_t>(u.cbegin(), u.cend()-1)   // vector constructed by coping elements of u except for the last

In [ ]:
%%cpp
vector<size_t> v(u.cbegin(), u.cend()-2) // direct initialization

To show that the elements are copied:

In [ ]:
%%cpp
L(u), L(v)

## Vector Contraction

We can clear all elements of a vector using the [`clear`](https://en.cppreference.com/w/cpp/container/vector/clear.html) method:
```cpp
void clear();
```

In [ ]:
%%cpp
vector<size_t> v {1, 2, 3};
v.clear();
v

To erase only some of the elements, use the [`erase`](https://en.cppreference.com/w/cpp/container/vector/erase.html) method:

::::{code} cpp
:linenos:
iterator erase( const_iterator pos );
iterator erase( const_iterator first, const_iterator last );
::::

which returns an iterator pointing to the element that follows the last removed element, or `end()` if the last element was erased.

In [ ]:
%%cpp
vector<size_t> v {1, 2, 3};
auto it=v.erase(v.cbegin()+1);
cout << format("Element before {} was removed.\n", *it);
v

::::{seealso} Constant iterator as position

The method takes the position in the form of a constant [iterator](https://en.cppreference.com/w/cpp/iterator.html), which is a generalization of pointers to the elements of the container:
- [`u.cbegin()`](https://en.cppreference.com/w/cpp/container/vector/begin.html)`+1` is the position of the second element.
- `u.cbegin()+i` is the position of the `(i+1)`-st element for any non-negative integer `i` smaller than `u.size()`. For containers that do not occupy continuous memory such as `std::list`, we may to use [`std::advance`](https://en.cppreference.com/w/cpp/iterator/advance.html)`(u, i)` instead.
  ```cpp
  template< class InputIt, class Distance > void advance( InputIt& it, Distance n );
  template< class InputIt, class Distance > constexpr void advance( InputIt&    it, Distance n );
  ```
- [`u.cend()-i`](https://en.cppreference.com/w/cpp/container/vector/end.html) gives the position of the `i`-th last element for positive integer `i` no larger than `u.size()`.
- `pos - u.cbegin()` or [`distance`](https://en.cppreference.com/w/cpp/iterator/distance.html)`(u.cbegin(), pos)` calculates how far an iterator `pos` of `u` is from its first element.
- `u.cend()-pos` or `distance(u.cend(), pos)` calculates how far an iterator `pos` of `u` is from the element one past the last element of `u`.

::::

In [ ]:
%%cpp
vector<size_t> v {1, 2, 3, 4};
auto it=v.cbegin();
advance(it, 3);
it=v.erase(it);
advance(it, -2);
it=v.erase(it);
cout << format("Elements before and after {} removed.\n", *it);
v

In [ ]:
%%cpp
it-v.cbegin()

In [ ]:
%%cpp
distance(v.cbegin(), it)

In [ ]:
%%ai
Explain in one line the difference between begin vs cbegin, and end vs cend for std::vector.

We can also erase a range of elements by specifying the `first` and `last` (exclusive) positions:

In [ ]:
%%cpp
vector<size_t> v {1, 2, 3};
v.erase(v.cbegin()+1, v.cend());
v

## Vector Expansion

### Insert at the Back

There are various ways to append an element to a `std::vector`.

One way is to use [`push_back()`](https://en.cppreference.com/w/cpp/container/vector/push_back), which have two overloads:

::::{code} cpp
:linenos:
void push_back( const T& value );
void push_back( T&& value );
::::

In [ ]:
%%cpp
vector<vector<size_t> > u;
vector<size_t> v {0,1,2};
u.push_back(v);
u

The above invokes the first overload since the rvalue reference in the second overload cannot bind to the lvalue `v`.

::::{exercise}
:label: ex:push_back1

Is `v` copied or moved to `u`?

::::

YOUR ANSWER HERE

Observe that `v` and `u[0]` start at different locations:

In [ ]:
%%cpp
LB(v), LB(u[0])

Modifying `v` does not affect `u`:

In [ ]:
%%cpp
v[0]=2;
u

`v` can be added to `u` again:

In [ ]:
%%cpp
u.push_back(v);
u

As a comparison, the following invokes the second overload by casting `v` as an rvalue using `std::move`:

In [ ]:
%%cpp
vector<vector<size_t> > u;
vector<size_t> v {0,1,2};
u.push_back(std::move(v));
u.push_back(v);
u

Note that `v` is cleared after the move. To show that the elements are moved:

In [ ]:
%%cpp
vector<vector<size_t> > u;
vector<size_t> v {0,1,2};
L(v);
u.push_back(std::move(LB(v)));
LB(u[0]), L(u), L(v)

Even though `v` and `u[0]` have different locations, their elements begin at the same location. In essense, the [move constructor](#move-constructor) is invoked to move the elements from `v` to a new vector in `u`.

We can also add an rvalue directly without creating an extra variable:

In [ ]:
%%cpp
vector<vector<size_t> > u;
u.push_back(vector<size_t>{0,1,2});
u

This can be further simplified:

In [ ]:
%%cpp
vector<vector<size_t> > u;
u.push_back({0,1,2});
u

::::{exercise}
:label: ex:push_back2

Explain how different constructors are invoked in the above call to `push_back`.

::::

YOUR ANSWER HERE

Indeed, we can further simplify the code as follows:

In [ ]:
%%cpp
vector<vector<size_t> > u;
u.emplace_back((initializer_list<size_t>){0,1,2});
u

::::{caution}

Both `u.emplace_back({0,1,2})` and `u.emplace_back(reinterpret_cast<std::initializer_list<size_t> >({0,1,2}))` fail. See [this post](https://stackoverflow.com/questions/24550924/emplacement-of-a-vector-with-initializer-list) for details.

::::

The constructed vector returned is exactly the vector added to the back:

In [ ]:
%%cpp
vector<vector<size_t> > u;
L(u.emplace_back((initializer_list<size_t>){0,1,2})), L(u[0])

Unlike `push_back`, [`emplace_back()`](https://en.cppreference.com/w/cpp/container/vector/emplace_back) constructs the vector [in-place](https://en.wikipedia.org/wiki/In-place_algorithm), i.e., the constructed vector is at the same location as the added vector:

```cpp
template< class... Args > reference emplace_back( Args&&... args );
```

`...` is a packing operator that declares a [pack](https://en.cppreference.com/w/cpp/language/parameter_pack.html) of parameters. This allows a variable number of arguments to be forwarded from `emplace_back` to the different overloads of the constructors for vector:

In [ ]:
%%cpp
vector<vector<size_t> > u;
u.emplace_back();          // forward to the default constructor
u

In [ ]:
%%cpp
vector<vector<size_t> > u;
u.emplace_back(2);         // forward to the fill constructor
u.emplace_back(3,4);
u

In comparison, Python uses `*` as the packing operator such as `*args` in `print`:

In [ ]:
print?

::::{exercise}
:label: ex:extend

Complete the second overload of the function `splice` to extend `u` with by copying elements from `v` to the end of `u`. If `v` is an rvalue, the elements should be moved instead, after which `v` should become emtpy.

::::

In [ ]:
%%cpp
template <class T>
void splice(vector<T> &u, const vector<T> &v) {
    for (auto vi : v) u.push_back(vi);
}

In [ ]:
%%cpp
template <class T>
void splice(vector<T> &u, vector<T> &&v) {
    /*
    # REPLACE THE ENTIRE COMMENT WITH YOUR CODE #
    */
}

In [ ]:
%%cpp
// test
vector<vector<size_t> > u {{1,2,3}};
vector<vector<size_t> > v {{4,5,6}};
splice(u, std::move(v));
splice(u, v);
splice(u, u);
u // ... { { 1, 2, 3 }, { 4, 5, 6 }, { 1, 2, 3 }, {} }

### Insertion

Elements can be inserted at different positions in a vector using the [`std::insert`](https://en.cppreference.com/w/cpp/container/vector/insert.html) method:

::::{code} cpp
:linenos:
iterator insert( const_iterator pos, const T& value );
iterator insert( const_iterator pos, T&& value );
iterator insert( const_iterator pos, size_type count, const T& value );
template< class InputIt > iterator insert( const_iterator pos, InputIt first, InputIt last );
iterator insert( const_iterator pos, std::initializer_list<T> ilist );
::::

In [ ]:
%%cpp
vector<size_t> u {0,2,3};
auto it=u.insert(u.cbegin()+1, 1);
u

After the insertion, the position/iterator of the inserted element is also returned.

In [ ]:
%%cpp
cout << *it << " inserted.\n";
it

Note that insertion takes linear time even the element is inserted at the back:

In [ ]:
%%cpp
vector<size_t> u {0,2,3};
L(u[0]), L(u[1]), L(u[2]);
cout << *u.insert(u.cbegin()+1, 1) << " inserted.\n";
L(u[0]), L(u[1]), L(u[2]), L(u[3]);
u

Observe that the locations of the elements are changed after insertion. In comparison, `push_back` normally does not change the locations of the existing elements:

In [ ]:
%%cpp
u.push_back(4);
L(u[0]), L(u[1]), L(u[2]), L(u[3]), L(u[4]);

`push_back` has an amortized constant time complexity because existing elements need to be moved only when there is no more free memory at the back of the vector.

The following behaves like the [fill constructors](#fill-constructors) that fill the vector with multiple copies of the same value:

In [ ]:
%%cpp
vector<size_t> u {1,2,3};
cout << format("Three {}'s added at the beginning.\n", *u.insert(u.cbegin(), 3, 0));
u

The following behaves like the [initializer-list constructor](#initializer-list-constructor):

In [ ]:
%%cpp
vector<size_t> u {1,2,3};
for (auto it=u.insert(u.cend(), {4,5,6}); it!=u.cend(); it++)
    cout << format("{} added.\n", *it);
u

We can also insert a sublist from another list:

In [ ]:
%%cpp
vector<size_t> u {1,2,3};
vector<double> v {4.1,5.2,6.3};
auto it=u.insert(u.cend(), v.cbegin(), v.cend()-1);
u

Note that elements are copied but not moved except when passed by rvalue reference:
```cpp
iterator insert( const_iterator pos, T&& value );
```

In [ ]:
%%cpp
vector<vector<size_t> > u {{1,2,3}};
vector<size_t> v {4,5,6};
u.insert(u.cbegin(), std::move(LB(v)));
LB(u[0]);
u.insert(u.cbegin(), v);
u

::::{exercise}
:label: ex:extend_pos

Complete the function `splice` to extend `u` by inserting elements from `v` at the position `pos` in `u`.  
- If `v` is an lvalue (`const vector<T>&`), the elements should be copied.  
- If `v` is an rvalue (`vector<T>&&`), the elements should be moved, and `v` should become empty afterward.  
 
The function should return an iterator pointing to the first inserted element in `u`. If no elements are inserted (i.e., `v` is empty), return `pos`.

::::

In [ ]:
%%cpp
template <class T>
typename vector<T>::iterator splice(vector<T> &u, typename vector<T>::const_iterator pos, const vector<T> &v) {
    /*
    # REPLACE THE ENTIRE COMMENT WITH YOUR CODE #
    */
}

In [ ]:
%%cpp
template <class T>
typename vector<T>::iterator splice(vector<T> &u, typename vector<T>::const_iterator pos, vector<T> &&v) {
    auto i = pos-u.cbegin();
    for (auto& vi : v) u.insert(pos++, std::move(vi));
    v.clear();
    return u.begin() + i;
}

In [ ]:
%%cpp
// test
vector<vector<size_t> > u {{1,2,3}};
vector<vector<size_t> > v {{4,5,6}};
auto it=splice(u, u.cbegin()+1, std::move(v));
it=splice(u, it-1, v);
it=splice(u, it, u);
u // ... { { 1, 2, 3 }, { 4, 5, 6 }, { 1, 2, 3 }, { 4, 5, 6 } }

In [ ]:
%%ai
Explain briefly why typename is used in:
--
void splice(vector<T> &u, typename vector<T>::const_iterator pos, const vector<T> &v);

## Linked List

The `splice` function in [](#ex:extend) and [](#ex:extend_pos) takes linear time as it inserts the elements of `v` to `u` one-by-one.

**Can we concatenate two vectors more efficiently?**

One idea is to link the end of one vector to the beginning of another. However, this isn't possible because `std::vector` stores its elements in a single, contiguous block of memory. This design enables constant-time access via indexing, i.e., `v[i]` is equivalent to `*(v + i)`, but it also means vectors cannot be physically joined without copying or reallocating their contents.

[](#code_combination2) uses another container [`std::list`](https://en.cppreference.com/w/cpp/container/list.html), which implements a [doubly linked list](https://en.wikipedia.org/wiki/Doubly_linked_list) data structure. Unlike `std::vector`, itt takes constant time to [`insert`](https://en.cppreference.com/w/cpp/container/list/insert.html#Complexity) or [`erase`](https://en.cppreference.com/w/cpp/container/list/erase.html#Complexity) a single element anywhere in the `std::list`. Furthermore, using the [`splice`](https://en.cppreference.com/w/cpp/container/list/splice.html) method from `std::list`, two lists can be concatenated efficiently in [constant time](https://en.cppreference.com/w/cpp/container/list/splice.html#Complexity) by modifying the end of one list to link to the beginning of another list:

::::{code} cpp
:linenos:
void splice( const_iterator pos, list& other );
void splice( const_iterator pos, list&& other );
void splice( const_iterator pos, list& other, const_iterator it );
void splice( const_iterator pos, list&& other, const_iterator it );
void splice( const_iterator pos, list& other, const_iterator first, const_iterator last );
void splice( const_iterator pos, list&& other, const_iterator first, const_iterator last );
::::

In [ ]:
%%cpp
list<int> a{1,2,3}, b{4,5,6,7};
a.splice(a.cend(), b);
a

All elements of `b` are now moved (*not copied*) to `a`:

In [ ]:
%%cpp
b

However, these come at the cost of linear time lookup.

In [ ]:
%%cpp
list<int> a{1,2,3,4,5,6,7};
// a[5];            // fails: type 'list' does not provide a subscript operator
auto it=a.cbegin();
// a.cbegin() + 5;  // fails: invalid operands to binary expression
for (auto i=5; i>0; i--) it++;  // `advance(it, 5)`
*it

As another example, consider the problem of enumerating the permutations in combinatorics:

::::{prf:definition}

Given a collection of $n\geq 0$ items $a_0,\dots,a_{n-1}$ and an integer $k\geq 0$, a [$k$-permutation of the $n$](https://en.wikipedia.org/wiki/Permutation#k-permutations_of_n) items is an ordered sequence 

$$
(a_{i_0},\dots,a_{i_{k-1}})
$$ 
of $k$ out of the $n$ objects for some choice of distinct indices $0\leq i_0,\dots,i_{k-1}<n$. An $n$-permutation of $n$ objects is called a permutation of $n$ objects.

::::

A simple way to generate permutations is by the following recurrence relation.

::::{prf:proposition}

A $k$-permutation of $n$ satisfies the recurrence relation for $1\leq k \leq n$:  
- Removing the first element of a $k$-permutation of $n$ objects gives a different $(k-1)$-permutation of the remaining $n-1$ objects.

$$ (a_{i_0}, \underbrace{a_{i_1},\dots,a_{i_{k-1}}}_{\text{\scriptsize $(k-1)$-permutation of $a_{i_1},\dots,a_{i_{k-1}}$.}\kern-5em} ) $$

- Reversing the above removal process gives a way of constructing a $k$-permutation from a $(k-1)$-permutation.  
  E.g., the permutations of $1,2,3$ can be constructed as follows:

$$[\overbrace{({\color{red} 1}, {\color{blue}{2, 3}}), ({\color{red} 1}, {\color{blue}{3, 2}})}^{\text{\scriptsize prepend 1 to permutations of $2,3$.} }, \overbrace{({\color{red} 2}, {\color{blue}{1, 3}}), ({\color{red} 2}, {\color{blue}{3, 1}})}^{\text{\scriptsize prepend 2 to permutations of $1,3$.} }, \overbrace{({\color{red} 3}, {\color{blue}{1, 2}}), ({\color{red} 3}, {\color{blue}{2, 1}})}^{\text{\scriptsize prepend 3 to permutations of $1,2$.} }]$$

::::

We can represent each permutation as a vector. The following implements the recursion:

In [ ]:
%%cpp
void permutation_helper(const vector<size_t> &items, int k, 
                        vector<size_t> &current, 
                        vector<vector<size_t> > &result) {
    if (k == 0) result.push_back(current);
    else
        for (size_t i = 0; i < items.size(); ++i) {
            vector<size_t> remaining = items;
            remaining.erase(remaining.cbegin() + i);
            current.push_back(items[i]);
            permutation_helper(remaining, k - 1, current, result);
            current.pop_back();
        }
}

In the above function:

- `current` keeps track of the beginning of the permutations to be built.
- `k` is the number of elements that need to be appended to `current` to complete each desired permutation.
- `items` is the remaining elements to choose from.
- `result` is a vector to store all completed permutations.

::::{note} Isn't it more efficient to use `std::list` instead of `std::vector` for `result`?
:class: dropdown

No, because `result` is generated by appending `current`, which can be done in amortized constant time for `std::vector`.

::::

The desired function that computes and returns the permutations can be a thin wrapper of the helper function:

In [ ]:
%%cpp
std::vector<vector<size_t> > permutation(const vector<size_t> & items, int k) {
    vector<vector<size_t> > result;
    vector<size_t>  current;

    if (k >= 0 && k <= items.size()) {
        permutation_helper(items, k, current, result);
    }
    return result;
}

permutation(vector<size_t> {1,2,3}, 2)

::::{note} Why not check the range of $k$ in the helper function instead?
:class: dropdown

This is because the check only needs to be done once. If it were placed inside the helper function, the check would be executed repeatedly in every recursive call.

::::

To make $k$ an optional argument that defaults to $n$:

In [ ]:
%%cpp
std::vector<vector<size_t> > permutation(const vector<size_t> &items) {
    return permutation(items, items.size());  // default to full length
}

permutation(vector<size_t> {1,2,3})

::::{note} Why is it better to overload the function than to give a default argument to `k`?
:class: dropdown

The default argument `items.size()` is not allowed since it is not a constant. Setting `k` to a value outside the valid range, say `-1`, is not ideal either because calling the function with `k=-1` will no longer be regarded an invalid.

::::

There is an inefficiency in the above program: Erasing an element from a vector is linear time, but removing an element from a list is constant time.

In [ ]:
%%ai
Explain briefly why `remaining.erase(it);` is more efficient if `remaining`
is a `std::list` instead of `std::vector`.

The following implementation of permutation operates directly on a list. No helper function is needed, since list merging and element removal are efficient.

In [ ]:
%%cpp
std::list<list<size_t> > permutation(const list<size_t> &items, int k) {
    list<list<size_t> > output;
    if (k == 0) {
        output.push_back(list<size_t> {});
    } else {
        list<size_t>  remaining = items;
        for (auto it = remaining.cbegin(); it != remaining.cend(); ++it) {
            auto i = *it;
            it=remaining.erase(it);
            list<list<size_t> > perm_seq = permutation(remaining, k - 1);
            it=remaining.insert(it, i);
            for (auto& perm : perm_seq) {
                perm.push_front(i);
            }
            output.splice(output.cend(), perm_seq);
        }
    }
    return output;
}

permutation(list<size_t> {1,2,3}, 2)

In [ ]:
%%cpp
std::list<list<size_t> > permutation(const list<size_t> &items) {
    return permutation(items, items.size());  // default to full length
}

permutation(list<size_t> {1,2,3})